In [1]:
import jupyter_dash as jd
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State
from dash import dash_table as dt
import dash_bootstrap_components as dbc

tableData=[{'Company Name':'names', 'CIK Number':'ciks'}]

c:\users\aymen\appdata\local\programs\python\python39\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
external_stylesheets = [dbc.themes.CERULEAN]
app = jd.JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
# page Title
    dbc.Row(html.H1('Gold Mine')),
    
# dataTable Inputs
    dbc.Row([
            dbc.Col(
                dcc.Input(
                    id='cikInput',
                    type='number',
                    placeholder='CIK Number',
                    maxLength=10
                ),
                width={"size": 1, "offset": 1}
            ),
            dbc.Col(
                dcc.Input(
                    id='nameInput',
                    type='text',
                    placeholder='Company Name',
                ),
                width={"size": 1}
            ),
            dbc.Col(
                dcc.Dropdown(
                    options=[
                        {'label':'10-K', 'value':'10-k'},
                        {'label':'10-Q', 'value':'10-q'}],
                    value= '10-k',
                    id='docTypes',
                    multi=True),
                width={"size": 2}
            ),
#from date, might switch to YYYYMMDD
            dbc.Col(
                dcc.DatePickerSingle(
                    id='fromDate',
                    display_format='YYYY-MM-DD',
                    placeholder='From'
                ),
                width={"size": 1}
            ),
#to date, might switch to YYYYMMDD
            dbc.Col(
                dcc.DatePickerSingle(
                id='toDate',
                display_format='YYYY-MM-DD',
                placeholder='To'
                ),
            width={"size": 1}
            ),
# owner Exclude/Include Check
            dbc.Col(
                dcc.Checklist(
                    options=[
                        {'label': 'Exclude', 'value': 'exclude'},
                        {'label': 'Include', 'value': 'include'}
                    ],
                    value=['exclude'],
                    id='ownerCheck'
                ),
            width={"size": 1, 'offset':1}
            ),    
        ]),
#     html.Div(id='inputList'),
#BUTTON ALERT OHHH FUCK
    html.Button('Submit', id='submit-val', n_clicks=0),
# DataTable showing Options
    dt.DataTable(
        id='table',
        columns=[
            {"name": 'CIK Number', "id": 'CIK Number'},
            {"name": 'Company Name', "id": 'Company Name'}
        ],
        data=tableData,
    )
])


@app.callback(
#     Output(component_id='inputList', component_property='children'),
    Output('table', 'data'),
    Input('submit-val', 'n_clicks'),
    State('cikInput', 'value'),
    State('nameInput', 'value'),
    State('docTypes', 'value'),
    State('fromDate', 'date'),
    State('toDate', 'date'),
    State('ownerCheck', 'value')
)
def inputList(clicks, cik, name, doc, fr, to, owner):
#     print(clicks)
    import requests
    from bs4 import BeautifulSoup
    endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

    headers = {'User-Agent':'ModernMerchants AdminContact@https://modernmerchants.squarespace.com'}

    #variable inputs
    cikInput=cik
    typeInput=doc
    startInput=fr
    datebInput=to
    companyInput=name
    ownerInput=owner


    param_dict={
        'action':'getcompany',
        'CIK': cikInput,
        'type': typeInput,
        'dateb':'20190101',
        'owner': ownerInput,
        'start': startInput,
        'output':'atom',
        'count':'100',
        'xbrl_type':'',
        'company': companyInput
    }
    response=requests.get(url=endpoint, params=param_dict, headers=headers)
    #print status code
    if response.status_code != 200:
        print('request error')
#     print(response.url)
###PART 2 OF PROGRAM
    soup = BeautifulSoup(response.content, parser='lxml')
    ciks=[x.text for x in soup.findAll('cik')]
    names=[x.text for x in soup.findAll('name')]
    names.remove('Webmaster')

    choices=pd.DataFrame({ 'Company Name':names, 'CIK Number':ciks})
#     print(choices.head())

    choiceInput=0
    cik=choices['CIK Number'][choiceInput]
    name=choices['Company Name'][choiceInput]
    tableData=[{'CIK Number': str(numberoni), 'Company Name': str(nameroni)} for nameroni, numberoni in [(nameroni, numberoni) for nameroni in names for numberoni in ciks]]
    return tableData

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
